In [1]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2025
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [2]:
!pip install --upgrade cdpcli 

In [3]:
# Import inference-related libraries
import httpx
import json
import time
import os
import subprocess
from urllib.parse import urlparse, urlunparse
from typing import Optional, Dict, Any, List

In [4]:
# Used while configuring CDP credentials config
import getpass
from llmopsUtils import *

CDP config function defined!


In [5]:
### CONNECT TO REMOTE REGISTRY FOR MODEL DOWNLOAD

In [6]:
# Configure CDP control plane credentials
access_key_id = getpass.getpass("Enter your CDP access key ID: ")
private_key = getpass.getpass("Enter your CDP private key: ")
configure_cdp("cdp_access_key_id", access_key_id)
configure_cdp("cdp_private_key", private_key)

Enter your CDP access key ID:  ········
Enter your CDP private key:  ········


In [7]:
ENVIRONMENT_NAME = "pdf-aw-cdp-env"

CDP_TOKEN = get_ums_jwt_token()
print("CDP TOKEN: ", CDP_TOKEN)

CDP TOKEN:  eyJraWQiOiIzYzhlNzA3OTEyZmI0NTA1ODE3NzE3YzMyOTU4MmQwMTFjYjlmNTAwIiwidHlwIjoiSldUIiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJwYXVsZGVmdXNjbyIsImF1ZCI6Imh0dHBzOi8vZGUuYTQ2NS05cTRrLmNsb3VkZXJhLnNpdGUiLCJpc3MiOiJodHRwczovL2NvbnNvbGVhdXRoLmNkcC5jbG91ZGVyYS5jb20vNTU4YmMxZDItODg2Ny00MzU3LTg1MjQtMzExZDUxMjU5MjMzIiwiZ3JvdXBzIjoiY2RwX3NhbmRib3hfd29ya2Vyc193dyBjb21wLWNsdXN0ZXJzLXRlc3RzLWF3LWNkcC1hZG1pbi1ncm91cCBzYW5kYm94LWRlZmF1bHQtc2UtYWRtaW4gY29tcC1jbHVzdGVycy10ZXN0cy1hdy1jZHAtdXNlci1ncm91cCBwYXVsLXByb2pzLW5vdi0xMS1hdy1jZHAtYWRtaW4tZ3JvdXAgcGRmLXNkYngtYXctY2RwLXVzZXItZ3JvdXAgcGRmLXNkYngtYXctY2RwLWFkbWluLWdyb3VwIHBhdWwtcHJvanMtbm92LTExLWF3LWNkcC11c2VyLWdyb3VwIF9jX2RmX3B1Ymxpc2hfNzUwZjNjNjAgX2NfbWxfdXNlcnNfMzE4ZTBiMmEgX2NfbWxfYWRtaW5zXzVkMzU1MThhIF9jX21sX2FkbWluc19mNTU1YWE2IF9jX2RmX3ZpZXdfZmY3NzRhZTAgX2NfZW52X2Fzc2lnbmVlc18yMThhMDVkZiBfY19tbF9hZG1pbnNfNDBkMTljMzAgX2NfbmlmaV9yZWdpc3RyeV9hZG1pbnNfZmY3NzRhZSBfY19tbF9hZG1pbnNfNmI4MjNjZWIgX2NfbWxfYWRtaW5zXzFlM2RhNzVlIF9jX2NtX2FkbWluc18yZjAwOWYwZiBfY

In [8]:
REGISTRY_ENDPOINT = get_registry_endpoint(ENVIRONMENT_NAME)
print("REGISTRY ENDPOINT: ", REGISTRY_ENDPOINT)

REGISTRY ENDPOINT:  https://modelregistry.ml-ef9ae082-698.pdf-aw-c.a465-9q4k.cloudera.site


In [9]:
REGISTERED_MODEL_NAME = "sparkml-clf"

model_details = get_model_details(REGISTRY_ENDPOINT, REGISTERED_MODEL_NAME, CDP_TOKEN)
print(model_details)

{'created_at': '2025-12-19T21:28:08.129Z', 'creator': {'user_name': 'pauldefusco'}, 'id': 'cw6b-8szi-rtdx-py4j', 'name': 'sparkml-clf', 'tags': None, 'updated_at': '2025-12-19T21:28:08.129Z', 'visibility': 'private'}


In [10]:
MODEL_VERSION = get_most_recent_model_version(REGISTRY_ENDPOINT, model_details['id'], CDP_TOKEN)

In [11]:
MODEL_ID = model_details['id']
print(MODEL_ID)
print(MODEL_VERSION)

cw6b-8szi-rtdx-py4j
1


In [12]:
MODEL_INFO = list_model_info(REGISTRY_ENDPOINT, MODEL_ID, CDP_TOKEN)
print(MODEL_INFO)

{'created_at': '2025-12-19T21:28:08.129Z', 'creator': {'user_name': 'pauldefusco'}, 'id': 'cw6b-8szi-rtdx-py4j', 'model_versions': [{'artifact_uri': 's3a://pdf-aw-buk-aec7c095/data/modelregistry/cw6b-8szi-rtdx-py4j/fkn7-1gh1-xxii-bhz5/model.tar.gz', 'created_at': '2025-12-19T21:28:08.130Z', 'metadata': {'mlflowMetadata': {'experiment_id': 'iusf-b1s2-pfau-x3tz', 'metrics': [], 'params': [{'key': 'framework', 'value': 'spark->sklearn->onnx'}, {'key': 'model_type', 'value': 'logistic_regression'}], 'run_id': 'pm9h-dt6u-sgoc-ugn3', 'tags': [{'key': 'engineID', 'value': 'ttjsal8n09pnqag6'}, {'key': 'mlflow.log-model.history', 'value': '[{"run_id": "pm9h-dt6u-sgoc-ugn3", "artifact_path": "spark-model", "utc_time_created": "2025-12-19 21:27:26.830348", "flavors": {"spark": {"pyspark_version": "3.5.1", "model_data": "sparkml", "code": null, "model_class": "pyspark.ml.pipeline.PipelineModel"}, "python_function": {"loader_module": "mlflow.spark", "python_version": "3.10.17", "data": "sparkml", "

In [13]:
latestVersionURI = MODEL_INFO['model_versions'][-1]['artifact_uri']

In [14]:
latestVersionURI

's3a://pdf-aw-buk-aec7c095/data/modelregistry/cw6b-8szi-rtdx-py4j/fkn7-1gh1-xxii-bhz5/model.tar.gz'

In [15]:
### DOWNLOAD MODEL TO LOCAL

In [16]:
download_model(latestVersionURI)

Bucket: pdf-aw-buk-aec7c095
Key: data/modelregistry/cw6b-8szi-rtdx-py4j/fkn7-1gh1-xxii-bhz5/model.tar.gz
Download successful!
Extraction complete!


In [17]:
### REGISTER THE MODEL LOCALLY

In [19]:
import mlflow
import torch
import numpy as np
import onnx
import onnxruntime as ort
import onnxmltools
from onnxmltools.convert.common.data_types import FloatTensorType

In [20]:
REGISTERED_MODEL_NAME_ONNX = "sparkml-clf"
onnx_path = "/home/cdsw/onnx_model" + "/model.onnx"
with mlflow.start_run() as run:    
    # Log the ONNX model to MLflow with input example and signature  
    print("Logging ONNX model to MLflow...")
    onnx_model = onnx.load(onnx_path)
    mlflow.onnx.log_model(
        onnx_model,
        "sparkml-clf",
        registered_model_name=f"{REGISTERED_MODEL_NAME_ONNX}"
    )
    
    print("Models logged successfully to MLflow!")
    print(f"MLflow Run ID: {run.info.run_id}")
    print("✓ Models logged with input examples and signatures!")

Creating run for experiment_id: 0, user_id: cdsw, run_name: None
No experiment set using default experiment.Please set experiment using mlflow.set_experiment('<your experiment name>') to avoid using default experiment.


Logging ONNX model to MLflow...


2025/12/19 22:59:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'sparkml-clf'.
experiment id 9k0x-srg5-u1z0-lhyc 
2025/12/19 22:59:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: sparkml-clf, version 1


Models logged successfully to MLflow!
MLflow Run ID: bpxm-ig5q-t1ec-ggeh
✓ Models logged with input examples and signatures!


Created version '1' of model 'sparkml-clf'.


In [8]:
### CONNECT TO LOCAL ENVIRONMENT'S INFERENCE SERVICE FOR MODEL DEPLOYMENT

In [21]:
ENVIRONMENT_NAME = "prod-cdp-env"

CDP_TOKEN = get_ums_jwt_token()
print("CDP TOKEN: ", CDP_TOKEN)

CDP TOKEN:  eyJraWQiOiIzYzhlNzA3OTEyZmI0NTA1ODE3NzE3YzMyOTU4MmQwMTFjYjlmNTAwIiwidHlwIjoiSldUIiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJwYXVsZGVmdXNjbyIsImF1ZCI6Imh0dHBzOi8vZGUuYTQ2NS05cTRrLmNsb3VkZXJhLnNpdGUiLCJpc3MiOiJodHRwczovL2NvbnNvbGVhdXRoLmNkcC5jbG91ZGVyYS5jb20vNTU4YmMxZDItODg2Ny00MzU3LTg1MjQtMzExZDUxMjU5MjMzIiwiZ3JvdXBzIjoiY2RwX3NhbmRib3hfd29ya2Vyc193dyBjb21wLWNsdXN0ZXJzLXRlc3RzLWF3LWNkcC1hZG1pbi1ncm91cCBzYW5kYm94LWRlZmF1bHQtc2UtYWRtaW4gY29tcC1jbHVzdGVycy10ZXN0cy1hdy1jZHAtdXNlci1ncm91cCBwYXVsLXByb2pzLW5vdi0xMS1hdy1jZHAtYWRtaW4tZ3JvdXAgcGRmLXNkYngtYXctY2RwLXVzZXItZ3JvdXAgcGRmLXNkYngtYXctY2RwLWFkbWluLWdyb3VwIHBhdWwtcHJvanMtbm92LTExLWF3LWNkcC11c2VyLWdyb3VwIF9jX2RmX3B1Ymxpc2hfNzUwZjNjNjAgX2NfbWxfdXNlcnNfMzE4ZTBiMmEgX2NfbWxfYWRtaW5zXzVkMzU1MThhIF9jX21sX2FkbWluc19mNTU1YWE2IF9jX2RmX3ZpZXdfZmY3NzRhZTAgX2NfZW52X2Fzc2lnbmVlc18yMThhMDVkZiBfY19tbF9hZG1pbnNfNDBkMTljMzAgX2NfbmlmaV9yZWdpc3RyeV9hZG1pbnNfZmY3NzRhZSBfY19tbF9hZG1pbnNfNmI4MjNjZWIgX2NfbWxfYWRtaW5zXzFlM2RhNzVlIF9jX2NtX2FkbWluc18yZjAwOWYwZiBfY

In [22]:
CAII_DOMAIN = get_caii_domain(ENVIRONMENT_NAME)

In [23]:
REGISTRY_ENDPOINT = get_registry_endpoint(ENVIRONMENT_NAME)
print("REGISTRY ENDPOINT: ", REGISTRY_ENDPOINT)

REGISTRY ENDPOINT:  https://modelregistry.ml-0ddb09fe-c77.prod-cdp.a465-9q4k.cloudera.site


In [24]:
REGISTERED_MODEL_NAME = "sparkml-clf"

model_details = get_model_details(REGISTRY_ENDPOINT, REGISTERED_MODEL_NAME, CDP_TOKEN)
print(model_details)

{'created_at': '2025-12-19T22:59:08.167Z', 'creator': {'user_name': 'pauldefusco'}, 'id': 'nk5u-0dxy-223b-pgxr', 'name': 'sparkml-clf', 'tags': None, 'updated_at': '2025-12-19T22:59:08.167Z', 'visibility': 'private'}


In [25]:
MODEL_VERSION = get_most_recent_model_version(REGISTRY_ENDPOINT, model_details['id'], CDP_TOKEN)

In [26]:
MODEL_ID = model_details['id']
print(MODEL_ID)
print(MODEL_VERSION)

nk5u-0dxy-223b-pgxr
1


In [27]:
ENDPOINT_NAME = "spark-clf-endpoint"
deploy_model_to_caii(CAII_DOMAIN, CDP_TOKEN, MODEL_ID, MODEL_VERSION, ENDPOINT_NAME)

Deployed spark-clf-endpoint successfully!


In [18]:
# Must return True before we go on to the next step
ready = endpoint_is_ready(CAII_DOMAIN, CDP_TOKEN, ENDPOINT_NAME)
print(ready)

False


In [19]:
BASE_URL = get_endpoint_base_url(CAII_DOMAIN, CDP_TOKEN, ENDPOINT_NAME)
print(BASE_URL)